In [1]:
import os
from string import Template
import subprocess

In [2]:
print(os.getcwd())

/home/lars/devel/pyversor


In [3]:
algebra_name = 'g312'

In [4]:
def cmakeliststxt(algebra_name):
    cmakelists = Template('''
cmake_minimum_required(VERSION 3.2)
project($algebra_name LANGUAGES CXX)

set(CMAKE_EXPORT_COMPILE_COMMANDS ON)

set(CMAKE_CXX_FLAGS "$${CMAKE_CXX_FLAGS} -std=c++14 -O3 -ftemplate-depth=2048 -fconstexpr-depth=2048")

include_directories(
  ../include
  ../include/versor
  ../include/versor/space
  ../include/versor/util
  )

find_package(pybind11)

pybind11_add_module($algebra_name
  $algebra_name.cpp
)                    
''')
    return cmakelists.substitute(algebra_name=algebra_name)

In [5]:
def gacpp(algebra_name, p, q=0, is_conformal=False):
    
    n_blades = 2**(p+q)
    
    ga_t = Template('vsr::algebra<vsr::metric<$p, $q, $is_conformal>, double>;')
    if is_conformal:
        ga_t = ga_t.substitute(p=p, q=q, is_conformal='true')
    else:
        ga_t = ga_t.substitute(p=p, q=q, is_conformal='false')
        
        
    mv_t = Template('''decltype(vsr::Multivector<ga_t, vsr::Basis<$blades>>() *
                      vsr::Multivector<ga_t, vsr::Basis<$blades>>());
''')
    mv_t = mv_t.substitute(blades=str([i for i in range(n_blades)]).replace('[','').replace(']',''))
    
    constructor = 'double, ' * n_blades
    constructor = constructor[:-2]
    
    
    cpp = Template('''
#include "pyversor/pyversor.h"

namespace py = pybind11;

using ga_t = $ga_t
using mv_t = $mv_t

PYBIND11_MODULE($algebra_name, m) {

  auto t = pyversor::def_multivector<mv_t>(m, "Multivector");
  t.def(py::init<$constructor>());
  pyversor::def_geometric_product<mv_t, mv_t>(t);

}
''')
    
    return cpp.substitute(algebra_name=algebra_name, ga_t=ga_t, mv_t=mv_t, constructor=constructor)

In [6]:
if not os.path.exists(algebra_name):
    os.mkdir(algebra_name)
    os.chdir(algebra_name)
    os.mkdir('build')

In [7]:
with open(algebra_name + '.cpp', 'w') as f:
    f.write(gacpp(algebra_name, 3,1, True))
    
with open('CMakeLists.txt', 'w') as f:
    f.write(cmakeliststxt(algebra_name))

In [8]:
os.chdir('build')

In [9]:
print(os.getcwd())

/home/lars/devel/pyversor/g312/build


In [10]:
subprocess.check_call(['cmake', '-GNinja', '..'])

0

In [11]:
subprocess.check_call(['cmake', '--build', '.'])

0